In [19]:
from pathlib import Path
import pandas as pd
import sys
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [20]:
df = pd.read_csv('CombinedData//final_clean_data.csv').sample(frac=1, random_state=42)
#df.columns
X = df.drop(columns=['patient_id', 'section', 'label', 'normal5', 'poor_quality5', 'normal1', 'poor_quality1', 'normal2', 'poor_quality2', 'normal3', 'poor_qualit3y', 'normal4', 'poor_quality4', 'quality1', 'quality2', 'quality3', 'quality4', 'quality5'], axis=1) 
y = df[['label']]

In [21]:
# if not (file_path.is_file() and file_path.suffix == '.csv'):
#     print("Error: The file condition is not met. Exiting program.")
#     sys.exit(1)

#define feature(X) and label(y) dataframes
#X includes all useful features
# X = pd.read_csv(file_path, usecols=lambda column: column not in ["patient_id", "section", "label"])
# # Y includes clinicions labels
# y = pd.read_csv(file_path, usecols= [2])


#remap your labels as follow to do 4 class classification
remap_SESCD_dict = { 0:0, 1:0, 2:0, 3:1, 4:1, 5:1, 6:2, 7:2, 8:2, 9:3, 10:3, 11:3, 12:3}
y['label'] = y['label'].replace(remap_SESCD_dict)
y.value_counts()


C:\Users\alexa\AppData\Local\Temp\ipykernel_19220\471729490.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['label'] = y['label'].replace(remap_SESCD_dict)


label
0.0      886
1.0      491
2.0      185
3.0       38
Name: count, dtype: int64

In [22]:
# use train_test_split to use 75% of your samples for training and 25% of your samples for testing
X_t, X_test, y_t, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)
y_t = y_t.values.ravel()
y_test = y_test.values.ravel()

In [23]:
models, best_param, y_pred_lst, y_prob_lst,scores, scores1, confs = [],[],[],[],[],[],[]
#use StratifiedShuffleSplit to do 5 folds cross validation on the training set.
#In each round keep 20% as the validation set (X_t: features for training set, y_t: labels for training set)
ss1 = StratifiedShuffleSplit(n_splits=5, test_size=0.20, random_state=0)

for t_index, v_index in ss1.split(X_t,y_t):
    X_train, y_train = X_t.iloc[t_index], y_t[t_index] 
    X_valid, y_valid = X_t.iloc[v_index], y_t[v_index]
    
    
    # Define the parameter grid
    param_grid = {
        'n_estimators': [50, 55, 60],
        #'max_depth': [None, 10, 20],
        'max_depth': [None, 20, 50],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [2, 4, 8],
        'max_features': [None, 'sqrt', 'log2']
    }
    
               
    
    cv = StratifiedKFold(n_splits=5, shuffle=False)
    rf = RandomForestClassifier()
    grid_search1 = GridSearchCV(estimator=rf, param_grid=param_grid, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=1)
    # fit to the training set
    grid_search1.fit(X_train, y_train)
    
    # Create a random forest classifier object with the best parameters
    RF_clf = RandomForestClassifier(n_estimators=grid_search1.best_params_['n_estimators'],
                                    max_depth=grid_search1.best_params_['max_depth'],
                                    min_samples_split=grid_search1.best_params_['min_samples_split'],
                                    min_samples_leaf=grid_search1.best_params_['min_samples_leaf'],
                                    max_features=grid_search1.best_params_['max_features'],
                                    random_state=0)

    print(grid_search1.best_params_['n_estimators'])
    print(grid_search1.best_params_['max_depth'])
    print(grid_search1.best_params_['min_samples_split'])
    print(grid_search1.best_params_['min_samples_leaf'])
    print(grid_search1.best_params_['max_features'])

    # Fit the model to the training data
    model = RF_clf.fit(X_train, y_train)

    # each round of cross validation will generate a ML model
    models.append(model)
    best_param.append(grid_search1.best_params_)
    y_pred = RF_clf.predict(X_test) #predicted labels for the test set
    y_pred_lst.append((RF_clf.predict(X_test)).reshape(len(y_test),1))

    # predict the probabilty scores for the test set
    y_prob = RF_clf.predict_proba(X_test)  
    y_prob_lst.append(y_prob)

    # check the models performance
    y_prob_positive_class = y_prob[:, 1]
    scores.append(roc_auc_score(y_test, y_prob, multi_class= 'ovr' ))
    scores1.append(grid_search1.best_score_)
    confs.append(confusion_matrix(y_test, y_pred))

y_pred_averages = (np.mean(y_pred_lst, axis=0)).flatten()



Fitting 5 folds for each of 243 candidates, totalling 1215 fits
55
None
10
8
log2
Fitting 5 folds for each of 243 candidates, totalling 1215 fits
60
50
10
8
log2
Fitting 5 folds for each of 243 candidates, totalling 1215 fits
60
20
10
4
log2
Fitting 5 folds for each of 243 candidates, totalling 1215 fits
60
None
2
4
log2
Fitting 5 folds for each of 243 candidates, totalling 1215 fits
50
None
2
2
sqrt
[array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.

In [25]:
print("label prediction averages across the 5 models")
print(y_pred_averages)
print("multiclass roc auc scores")
print(scores)
print("")
print(scores1)
print()
print(confs)


[[0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [1. ]
 [0.2]
 [0. ]
 [0.8]
 [1. ]
 [0. ]
 [0.4]
 [0. ]
 [0.8]
 [0. ]
 [0. ]
 [0. ]
 [0.8]
 [0. ]
 [0. ]
 [0. ]
 [1. ]
 [0. ]
 [1.4]
 [0. ]
 [0.8]
 [1.2]
 [0. ]
 [0.4]
 [1.2]
 [0.2]
 [1. ]
 [0.2]
 [1. ]
 [0.4]
 [0.2]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [1. ]
 [0. ]
 [0. ]
 [0. ]
 [0.8]
 [0. ]
 [0. ]
 [1. ]
 [1. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [2. ]
 [0. ]
 [0. ]
 [1. ]
 [1.4]
 [0.2]
 [1. ]
 [0.6]
 [0. ]
 [1. ]
 [0. ]
 [1. ]
 [0.4]
 [1. ]
 [0. ]
 [1. ]
 [1.8]
 [0. ]
 [0. ]
 [0.2]
 [1.6]
 [1. ]
 [0. ]
 [0. ]
 [0. ]
 [1. ]
 [0. ]
 [1. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0.2]
 [0. ]
 [0. ]
 [0.6]
 [0. ]
 [0. ]
 [0. ]
 [0.2]
 [1. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [2. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [1. ]
 [0. ]
 [0. ]
 [0.2]
 [1. ]
 [0. ]
 [0. ]
 [0.6]
 [0.8]
 [0. ]
 [1. ]
 [0. ]
 [0.6]
 [0.8]
 [0. ]
 [0. ]
 [1.8]
 [0. ]
 [0. ]
 [2. ]
 [0. ]
 [0. ]
 [0. ]
 [1. ]
 [0. ]
 [0. 